In [1]:
import pandas as pd
from selenium import webdriver
import time

### Getting the URLs to be scraped
* The csv used is a list created with crawl data. 
* Crawling the page I was able to get ~28% of all the existing location. This set of data included multiple 404 pages, that were not showing a 404 response code.
* To complete with all location, I've added all HREF found in location/city pages that were missing.

In [3]:
urls = pd.read_csv('urls_to_scrape.csv')

### Setting our DataFrames index:

Will set the index to be url_id, since the urls will be the primary key of my tables.

In [4]:
urls['url_id'] = urls['URL']
urls['url_id'].is_unique #checking if we have duplicated values

True

In [5]:
#settings urls as index
urls = urls.set_index('url_id')

### Labeling our urls
To get the protocols, domains and paths, the url will be splited twice ('://', '/')
* Protocol will let us know if there is any location in without SSL
* Path 1 should all be location (using another script I had already done the cleaning and boolean selections)
* Path 2 will be used to identify the city (scrap data for city is not 100% exact)
* Path 3 will be used to verify address

In [6]:
#'://'
split = pd.DataFrame()
split[['protocol','url_no_protocol']] = urls['URL'].str.split('://', expand = True, n = 1)
split2 = split['url_no_protocol']
#'/'
split_again = pd.DataFrame()
split_again[['domain','path1','path2','path3']] = split2.str.split('/',expand = True, n=3)

In [7]:
urls_labeled = pd.merge(split, split_again, left_index = True, right_index = True)
urls_labeled.head(3)

,protocol,url_no_protocol,domain,path1,path2,path3
url_id,,,,,,
https://breather.com/locations/boston/262-washington-2,https,breather.com/locations/boston/262-washington-2,breather.com,locations,boston,262-washington-2
https://breather.com/locations/boston/50-congress-543,https,breather.com/locations/boston/50-congress-543,breather.com,locations,boston,50-congress-543
https://breather.com/locations/boston/11-beacon-605,https,breather.com/locations/boston/11-beacon-605,breather.com,locations,boston,11-beacon-605


### Understanding our urls
Will use .describe() to se how many protocols, domains, cities and locations we have:

In [8]:
urls_labeled.describe()

,protocol,url_no_protocol,domain,path1,path2,path3
count,435,435,435,435,435,435
unique,1,435,1,1,10,435
top,https,breather.com/locations/washington-dc/dc-444-no...,breather.com,locations,new-york,1123-broadway-310
freq,435,1,435,435,144,1


We have:
* Protocol = 1
* Domain = 1
* Cities = 10
* Locations = 435

### Understanding the locations
Since protocol and domain are uniques, will use groupby to understand how many location there are in each city.

In [9]:
group_cities = urls_labeled.groupby('path2')
group_cities['path3'].describe()

,count,unique,top,freq
path2,,,,
boston,30,30,262-washington-1,1
chicago,28,28,1500-w-carroll-suite-1,1
london,33,33,43maidenlane3,1
los-angeles,35,35,1639-11th-210,1
montreal,31,31,2075-robertbourassa,1
new-york,144,144,211-e-43-2,1
ottawa,2,2,byward-market,1
san-francisco,67,67,south-park,1
toronto,41,41,40-university,1


# Lets begin getting our data
After taking a look to the kind of data we could grab from these urls, I've decided to get all these information for the analysis:
* amenities	: list of strings
* additional-in-room-service : list of strings with its values
* accommodates-text	: list of strings
* accommodates-numbers : list of integers
* Opening days : list of strings
* Opening hours : list of strings
* Good for : list of strings
* address : unique string
* city : unique string
* neighborhood : unique string
* rating : float
* rating-number : int
* prices : int

Now, I have two options when scraping the site:
1) Could get all this data in lists and string per urls, and then unnest the lists and change the data structure
2) Could identify exactly how I want to manipulate the data, and create the scipt to do it automatically.

Since I would prefer not to make to many requests with trial and error, its better if I get all the data at once, and then play with it (note that this scraping will be done once, so there is no need for it to be automated).

## Creating our DataFrames:

To better store and understand the data, I'll keep using URL as our primary key, what will connect all tables. I would definitelly keep everything in a big table, but will separate the information in 6 tables:

### Unique values
1) General table: url, address, city, neighborhood, rating, rating-number

### Boolean tables
2) Amenities table: url, and all possible amenities you could have as columns. Each URL will have a boolean result to se if its present or not.

3) Good for table: url, and all good for characteristics in the site as columns. Each URL will have a boolean result to se if its present or not.

### Columns with values
4) Additional in room service table: url, and all possible option avaiable in the site. Each URL wil have the price for each additional product. 

5) Prices table: url, price per hour, price per day. Each URL will have both price points.

6) Opening hours table: url, Opening Mon, Closing Mon, Opening Tue, Closing Tue, Opening Wed, Closing Wed, Opening Thu, Closing Thy, Opening Fri, Closing Fri, Opening Sat, Closing Sat, Opening Sun, Closing Sun. Each URL will have a time value. With this will be able to calculate total availability per timeperiod. 

## Lets set up our lists and disctionaries
For the scraping, I'll define 3 different types of data extraction:

1) data_showmore: These are the elements that might need a click to show more results. --values_with_showmore

2) data_values: There are the elements that have labels and values --values_with_values

3) data_text: These are the elements that are uniques, and have text (can be a unique list) --values_no_showmore

In [10]:
data_showmore = ['amenities', 'additional-in-room-service']
data_values = ['accommodates', 'opening-hours']
data_text = ['good-for', 'address', 'city', 'neighborhood', 'rating', 'rating-amount', 'prices']

text_xpaths = {'amenities': '''//ul[@data-e2e='amenities']//li//span''',
            'accommodates' : '''//ul[@data-e2e='accommodates']//li//span[1]''',
            'opening-hours' : '''//ul[@data-e2e='opening-hours']//li//span[1]''',
            'good-for' : '''//ul[@data-e2e='good-for'][1]//span''',
            'additional-in-room-service' : '''//div[@class='inub8q-0 isVhgx']//p[1]''',
            'address' : '''//div[@class='moss8l-0 bUznMw']''',
            'city' : '''//a//span[@class='rd1u8h-0 dLqEBT'][1]''',
            'neighborhood' : '''//a//span[@class='rd1u8h-0 dLqEBT'][2]''',
            'rating' : '''//span[@class='Rating__Rate-sc-1iav4yy-2 fzykSR']''',
            'rating-amount' : '''//span[@class='Rating__Count-sc-1iav4yy-3 hjXkay']''',
            'prices' : '''//div[@class='Price-izt15s-0 fEKEcE']'''}

values_xpaths = {'accommodates': '''//ul[@data-e2e='accommodates']//li//span[2]''',
                       'opening-hours': '''//ul[@data-e2e='opening-hours']//li//span[2]'''}
showmore_xpaths = {'amenities': '''//button[@class='Amenities__Button-sc-116xlka-0 fSHBKu x85bks-0 byiYrK s4i5irx-0 gsxtBT']''',
                    'additional-in-room-service': '''//button[@class='Services__Button-sc-1jbd96f-4 gYUVoh x85bks-0 byiYrK s4i5irx-0 gsxtBT']'''}

In [11]:
#all location urls to be scraped:
urls_to_scrape = urls['URL']
urls_to_scrape.shape

(435,)

### Getting our DataFrames columns!
First we need to see which values we need to get for our lists. 

In [12]:
amenities_per_url = []
aditional_in_room_service_per_url = []
accommodates_per_url = []
accommodates_per_url_v = []
opening_days_per_url = []
opening_hours_per_url = []
good_for_per_url = []
address_per_url = [] 
city_per_url = [] 
neighborhood_per_url = []
rating_per_url = []
rating_amount_per_url = []
prices_per_url = []

driver = webdriver.Chrome()

for url in urls_to_scrape:
    print(url)
    driver.get(url)
    time.sleep(2)
    #creating the lists for the values to be appended. These lists will be merged with the URL (should have the same dimensions)
    good_for = []
    address = []
    city = []
    neighborhood = []
    rating = []
    rating_number = []
    prices = []
    
    #Group1 lists
    amenities = []
    aditional_in_room_service = []
    
    #Group2 lists
    accommodates = []
    accommodates_values = []
    opening_days = []
    opening_hours = []
    
    #Group3 lists
    good_for = []
    address = [] 
    city = [] 
    neighborhood = []
    rating = []
    rating_amount = []
    prices = []
    
    #Group1 data extraction
    for data in data_showmore:
        showmore = driver.find_elements_by_xpath(showmore_xpaths[data])
        if len(showmore) > 0:
            showmore[0].click()
        texts = driver.find_elements_by_xpath(text_xpaths[data])
        
        #lists to separate the values in the elements in lists
        for text in texts:
            if data == 'amenities':
                amenities.append(text.text)
            else: 
                aditional_in_room_service.append(text.text)
    
    #Group2 data extraction
    for data in data_values:
        texts = driver.find_elements_by_xpath(text_xpaths[data])
        values = driver.find_elements_by_xpath(values_xpaths[data])
        for text in texts:
            if data == 'accommodates':
                accommodates.append(text.text)
            else: 
                opening_days.append(text.text)
        for value in values:
            if data == 'accommodates':
                accommodates_values.append(value.text)
            else: 
                opening_hours.append(value.text)
        
    #Group3 data extraction
    for data in data_text:
        texts = driver.find_elements_by_xpath(text_xpaths[data])
        if len(texts) > 0:
            for text in texts:
                if data == 'good-for':
                    good_for.append(text.text)
                elif data == 'address':
                    address.append(text.text)
                elif data == 'city':
                    city.append(text.text)
                elif data == 'neighborhood':
                    neighborhood.append(text.text)
                elif data == 'rating':
                    rating.append(text.text)
                elif data == 'rating-amount':
                    rating_amount.append(text.text)
                else:
                    prices.append(text.text)
        else:
            if data == 'good-for':
                    good_for.append('')
            elif data == 'address':
                    address.append('')
            elif data == 'city':
                    city.append('')
            elif data == 'neighborhood':
                    neighborhood.append('')
            elif data == 'rating':
                    rating.append('')
            elif data == 'rating-amount':
                    rating_amount.append('')
            else:
                    prices.append('')
        
        #lists to separate the values in the elements in lists
    
    # list to append the results per url and per column
    aditional_in_room_service_per_url.append(aditional_in_room_service)
    amenities_per_url.append(amenities)
    accommodates_per_url.append(accommodates)
    accommodates_per_url_v.append(accommodates_values)
    opening_days_per_url.append(opening_days)
    opening_hours_per_url.append(opening_hours)
    good_for_per_url.append(good_for)
    address_per_url.append(address)
    city_per_url.append(city) 
    neighborhood_per_url.append(neighborhood)
    rating_per_url.append(rating)
    rating_amount_per_url.append(rating_amount)
    prices_per_url.append(prices)

driver.close()
print('Scraping is done!')


https://breather.com/locations/boston/262-washington-2
https://breather.com/locations/boston/50-congress-543
https://breather.com/locations/boston/11-beacon-605
https://breather.com/locations/boston/262-washington-8
https://breather.com/locations/boston/11-beacon-1110


KeyboardInterrupt: 

In [42]:
primary_df = pd.DataFrame(urls_to_scrape)
primary_df.shape

(435, 1)

In [43]:
primary_df['amenities'] = amenities_per_url
primary_df['addons'] = aditional_in_room_service_per_url
primary_df['accommodates'] = accommodates_per_url
primary_df['accommodatesv'] = accommodates_per_url_v
primary_df['oppening_days'] = opening_days_per_url
primary_df['oppening_hours'] = opening_hours_per_url
primary_df['good_for'] = good_for_per_url
primary_df['address'] = address_per_url
primary_df['city'] = city_per_url
primary_df['neighborhood'] = neighborhood_per_url
primary_df['rating_score'] = rating_per_url
primary_df['rating_number'] = rating_amount_per_url
primary_df['prices'] = prices_per_url

In [45]:
primary_df.head(5)

,URL,amenities,addons,accommodates,accommodatesv,oppening_days,oppening_hours,good_for,address,city,neighborhood,rating_score,rating_number,prices
url_id,,,,,,,,,,,,,,
https://breather.com/locations/boston/262-washington-2,https://breather.com/locations/boston/262-wash...,"[Whiteboard (1), 65"" Flatscreen TV, Air condit...","[Folding Chair$13, Multipurpose Table$100, Pro...","[At tables, In lounge, Maximum total, Sq. ft.]","[8, 3, 11, 440]","[Mon - Thu, Fri, Sat - Sun]","[8:00 am - 9:00 pm, 8:00 am - 8:00 pm, Closed]","[Team Offsite, Team Meeting, Presentation, Cli...","[262 Washington Street, 4th Floor, Suite 402, ...",[Boston],[Downtown],[4.7],[29 Ratings],"[$57/hour, $452/day]"
https://breather.com/locations/boston/50-congress-543,https://breather.com/locations/boston/50-congr...,"[Whiteboard, Air conditioning, WiFi, Lobby sta...","[Folding Chair$13, Multipurpose Table$100, Pro...","[At tables, In lounge, Maximum total, Sq. ft.]","[4, 2, 6, 226]","[Mon - Fri, Sat, Sun]","[7:00 am - 8:00 pm, 8:00 am - 4:00 pm, Closed]","[Client Meeting, Team Meeting, Therapy / Consu...","[50 Congress Street, 5th Floor, Suite 543]",[Boston],[Downtown],[4.4],[395 Ratings],"[$49/hour, $392/day]"
https://breather.com/locations/boston/11-beacon-605,https://breather.com/locations/boston/11-beaco...,"[Whiteboards (2), 65"" Flatscreen TV, Air condi...","[Folding Chair$13, Multipurpose Table$100, Pro...","[At tables, In lounge, Maximum total, Sq. ft.]","[12, 2, 14, 485]","[Mon - Fri, Sat - Sun]","[8:00 am - 6:00 pm, Closed]","[Team Offsite, Team Meeting, Presentation, Cli...","[11 Beacon Street, 6th Floor, Suite 605]",[Boston],[Downtown],[4.6],[9 Ratings],"[$65/hour, $520/day]"
https://breather.com/locations/boston/262-washington-8,https://breather.com/locations/boston/262-wash...,"[Whiteboard (1), 65"" Flatscreen TV, Air condit...","[Folding Chair$13, Multipurpose Table$100, Pro...","[At tables, In lounge, Maximum total, Sq. ft.]","[12, 2, 14, 470]","[Mon - Thu, Fri, Sat - Sun]","[8:00 am - 9:00 pm, 8:00 am - 8:00 pm, Closed]","[Team Offsite, Team Meeting, Presentation, Cli...","[262 Washington Street, 8th Floor, Suite 801]",[Boston],[Downtown],[4.7],[11 Ratings],"[$59/hour, $472/day]"
https://breather.com/locations/boston/11-beacon-1110,https://breather.com/locations/boston/11-beaco...,"[Whiteboards (2), 65"" Flatscreen TV, Air condi...","[Folding Chair$13, Multipurpose Table$100, Pro...","[At tables, In lounge, Maximum total, Sq. ft.]","[18, 4, 22, 752]","[Mon - Fri, Sat - Sun]","[8:00 am - 6:00 pm, Closed]","[Team Offsite, Team Meeting, Presentation, Cli...","[11 Beacon Street, 11th Floor, Suite 1110]",[Boston],[Downtown],[4.8],[12 Ratings],"[$95/hour, $759/day]"


In [48]:
primary_df.to_csv('primary_df.csv')

Now that we saved the scraped data, we can start cleaning it. 

In [46]:
accommodates = primary_df['accommodatesv'].apply(pd.Series)
prices = primary_df['prices'].apply(pd.Series)
city = primary_df['city'].apply(pd.Series)
address = primary_df['address'].apply(pd.Series)
neighborhood = primary_df['neighborhood'].apply(pd.Series)
rating_score = primary_df['rating_score'].apply(pd.Series)
rating_number = primary_df['rating_number'].apply(pd.Series)

In [49]:
#Renaming the columns:
accommodates.columns = ['at_tables', 'in_lounge', 'maximum_total', 'sq_ft']
prices.columns = ['hourly_price','daily_price'] #CLEAN
city.columns = ['city']
address.columns = ['address']
neighborhood.columns = ['neighborhood']
rating_score.columns = ['rating_score'] #CLEAN
rating_number.columns = ['rating_number'] #CLEAN

In [50]:
#Cleaning accommodates table
accommodates['at_tables'] = pd.to_numeric(accommodates['at_tables'])
accommodates['in_lounge'] = pd.to_numeric(accommodates['in_lounge'])
accommodates['maximum_total'] = pd.to_numeric(accommodates['maximum_total'])
accommodates['sq_ft'] = pd.to_numeric(accommodates['sq_ft'])
accommodates.head(3)

,at_tables,in_lounge,maximum_total,sq_ft
url_id,,,,
https://breather.com/locations/boston/262-washington-2,8.0,3.0,11.0,440.0
https://breather.com/locations/boston/50-congress-543,4.0,2.0,6.0,226.0
https://breather.com/locations/boston/11-beacon-605,12.0,2.0,14.0,485.0


In [51]:
accommodates.describe()

,at_tables,in_lounge,maximum_total,sq_ft
count,424.000000,424.000000,424.000000,424.000000
mean,15.002358,4.099057,19.816038,649.924528
std,9.377176,2.300087,12.090249,377.185432
min,2.000000,0.000000,3.000000,100.000000
25%,8.000000,3.000000,10.000000,342.500000
50%,12.000000,4.000000,16.500000,595.000000
75%,20.000000,5.000000,27.000000,881.500000
max,48.000000,23.000000,75.000000,2242.000000


In [52]:
#CLEANING Rating_number
splited = pd.DataFrame()
splited[['ratings','remove']] = rating_number['rating_number'].str.split(' ', expand = True, n = 1)
ratings = pd.DataFrame(splited['ratings'])
#CLEANING Rating_score. We are removing "New Space" and change it with None.
criteria = rating_score['rating_score'] == 'New Space'
rating_score[criteria] = ""
ratings_all = ratings.merge(rating_score, left_index = True, right_index = True)
#ratings_all['rating_score']=ratings_all['rating_score'].str.replace('.',',')
ratings_all['ratings']=ratings_all['ratings'].str.replace(',','')
ratings_all['ratings'] = pd.to_numeric(ratings_all['ratings'])
ratings_all['rating_score'] = pd.to_numeric(ratings_all['rating_score'])
ratings_all.head(3)

,ratings,rating_score
url_id,,
https://breather.com/locations/boston/262-washington-2,29.0,4.7
https://breather.com/locations/boston/50-congress-543,395.0,4.4
https://breather.com/locations/boston/11-beacon-605,9.0,4.6


In [53]:
#Cleaning prices
criteria2 = prices['hourly_price'].str.contains('/day')
# the locations with mislocated prices are
wrong_price_loc = prices[criteria2].index

# creating a dictionary with the mislocated prices
mislocated_prices = {}
for url in wrong_price_loc:
    mislocated_prices[url] = prices.loc[url]['hourly_price']
prices.head(5)

In [58]:
# reasignin the prices to the write column
for url in mislocated_prices:
    prices.loc[url]['daily_price']=mislocated_prices[url]
# removing daily prices from the hourly price column
for url in mislocated_prices:
    prices.loc[url]['hourly_price']=""
prices.head(5)

,hourly_price,daily_price
url_id,,
https://breather.com/locations/boston/262-washington-2,$57/hour,$452/day
https://breather.com/locations/boston/50-congress-543,$49/hour,$392/day
https://breather.com/locations/boston/11-beacon-605,$65/hour,$520/day
https://breather.com/locations/boston/262-washington-8,$59/hour,$472/day
https://breather.com/locations/boston/11-beacon-1110,$95/hour,$759/day


In [59]:
# cleaning the text
prices.replace(regex={r'/hour':'', '/day':''},inplace=True)
prices.head(5)

,hourly_price,daily_price
url_id,,
https://breather.com/locations/boston/262-washington-2,$57,$452
https://breather.com/locations/boston/50-congress-543,$49,$392
https://breather.com/locations/boston/11-beacon-605,$65,$520
https://breather.com/locations/boston/262-washington-8,$59,$472
https://breather.com/locations/boston/11-beacon-1110,$95,$759


In [62]:
# removing money signs
prices.replace(regex={'$':'','£':''},inplace=True)
#because '$' is a regular expression, we need to replace again without regex.
prices['hourly_price']=prices['hourly_price'].str.replace('$','')
prices['daily_price']=prices['daily_price'].str.replace('$','')
prices.head(5)

,hourly_price,daily_price
url_id,,
https://breather.com/locations/boston/262-washington-2,57,452
https://breather.com/locations/boston/50-congress-543,49,392
https://breather.com/locations/boston/11-beacon-605,65,520
https://breather.com/locations/boston/262-washington-8,59,472
https://breather.com/locations/boston/11-beacon-1110,95,759


In [63]:
# removing ','
prices['hourly_price']=prices['hourly_price'].str.replace(',','')
prices['daily_price']=prices['daily_price'].str.replace(',','')
prices.head(5)

,hourly_price,daily_price
url_id,,
https://breather.com/locations/boston/262-washington-2,57,452
https://breather.com/locations/boston/50-congress-543,49,392
https://breather.com/locations/boston/11-beacon-605,65,520
https://breather.com/locations/boston/262-washington-8,59,472
https://breather.com/locations/boston/11-beacon-1110,95,759


In [64]:
# converting str into numbers
prices['hourly_price'] = pd.to_numeric(prices['hourly_price'])
prices['daily_price'] = pd.to_numeric(prices['daily_price'])

In [65]:
# merge prices with locations
price_analysis = prices.merge(urls_labeled[['path2']],left_index=True,right_index=True)
price_analysis.rename(columns={'path2': 'location'}, inplace = True)
price_analysis.head(5)

,hourly_price,daily_price,location
url_id,,,
https://breather.com/locations/boston/262-washington-2,57.0,452.0,boston
https://breather.com/locations/boston/50-congress-543,49.0,392.0,boston
https://breather.com/locations/boston/11-beacon-605,65.0,520.0,boston
https://breather.com/locations/boston/262-washington-8,59.0,472.0,boston
https://breather.com/locations/boston/11-beacon-1110,95.0,759.0,boston


In [69]:
# using page source data I know that the prices are assign to each countries currencies. 
# creating dictionary for the currencies. 
cities= [{"id":"8jLq7Yxb3R","name":"New York City","slug":"new-york","shortSlug":"nyc","lnglat":[-73.9910835,40.7358633],"defaultSearchRadius":25000,"timezone":"America/New_York","status":"active","creditValue":4500,"currencyCode":"USD","region":"New York","regionCode":"NY","country":"United States","countryCode":"US","sortOrder":1,"locationCount":149,"_vid":79,"createdAt":"2014-02-05T16:32:47.014Z","updatedAt":"2018-08-10T19:14:00.881Z"},{"id":"kMG2IsYtKL","name":"SF Bay Area","slug":"san-francisco","shortSlug":"sf","lnglat":[-122.404261,37.784486],"defaultSearchRadius":25000,"timezone":"America/Los_Angeles","status":"active","creditValue":4500,"currencyCode":"USD","region":"California","regionCode":"CA","country":"United States","countryCode":"US","sortOrder":2,"locationCount":65,"_vid":45,"createdAt":"2014-05-29T16:32:47.014Z","updatedAt":"2018-09-17T18:14:00.481Z"},{"id":"lIRf1YeVcA","name":"Boston","slug":"boston","shortSlug":"bos","lnglat":[-71.106884,42.370093],"defaultSearchRadius":25000,"timezone":"America/New_York","status":"active","creditValue":4500,"currencyCode":"USD","region":"Massachusetts","regionCode":"MA","country":"United States","countryCode":"US","sortOrder":3,"locationCount":32,"_vid":21,"createdAt":"2015-01-08T16:00:00.000Z","updatedAt":"2018-04-30T20:14:00.859Z"},{"id":"fTb7f2cerf","name":"Los Angeles","slug":"los-angeles","shortSlug":"la","lnglat":[-118.2436849,34.0522342],"defaultSearchRadius":50000,"timezone":"America/Los_Angeles","status":"active","creditValue":4500,"currencyCode":"USD","region":"California","regionCode":"CA","country":"United States","countryCode":"US","sortOrder":4,"locationCount":38,"_vid":39,"createdAt":"2015-06-26T18:53:59.775Z","updatedAt":"2018-06-29T18:14:01.592Z"},{"id":"Zn3FHU9Kej","name":"Chicago","slug":"chicago","shortSlug":"chi","lnglat":[-87.6297982,41.8781136],"defaultSearchRadius":50000,"timezone":"America/Chicago","status":"active","creditValue":4500,"currencyCode":"USD","region":"Illinois","regionCode":"IL","country":"United States","countryCode":"US","sortOrder":5,"locationCount":29,"_vid":17,"createdAt":"2015-06-26T18:53:59.816Z","updatedAt":"2018-08-30T18:14:00.484Z"},{"id":"jxCwZvcWuf","name":"Washington DC","slug":"washington-dc","shortSlug":"dc","lnglat":[-77.0368707,38.9071923],"timezone":"America/New_York","status":"active","creditValue":4500,"currencyCode":"USD","region":"District of Columbia","regionCode":"DC","country":"United States","countryCode":"US","sortOrder":6,"locationCount":27,"_vid":16,"createdAt":"2015-06-26T18:53:59.774Z","updatedAt":"2017-12-01T19:14:00.712Z"},{"id":"HEG8fNPoN3","name":"London","slug":"london","shortSlug":"lon","lnglat":[-0.1277583,51.5073509],"timezone":"Europe/London","status":"active","creditValue":3500,"currencyCode":"GBP","region":"England","regionCode":"England","country":"United Kingdom","countryCode":"GB","sortOrder":7,"locationCount":40,"_vid":29,"createdAt":"2015-06-26T18:53:59.802Z","updatedAt":"2018-03-12T17:14:00.878Z"},{"id":"x7bPMpzIvM","name":"Montreal","slug":"montreal","shortSlug":"mtl","lnglat":[-73.588368,45.503746],"defaultSearchRadius":25000,"timezone":"America/Montreal","status":"active","creditValue":2500,"currencyCode":"CAD","region":"Québec","regionCode":"QC","country":"Canada","countryCode":"CA","sortOrder":8,"locationCount":31,"_vid":40,"createdAt":"2013-10-15T15:36:34.294Z","updatedAt":"2018-02-26T17:14:00.073Z"},{"id":"e9447930a01e11e581aa851ef65f5f58","name":"Toronto","slug":"toronto","shortSlug":"tor","lnglat":[-79.38166,43.6525],"defaultSearchRadius":50000,"timezone":"America/Toronto","status":"active","creditValue":2500,"currencyCode":"CAD","region":"Ontario","regionCode":"ON","country":"Canada","countryCode":"CA","sortOrder":9,"locationCount":43,"_vid":34,"createdAt":"2015-12-11T15:50:32.779Z","updatedAt":"2018-06-23T03:14:00.462Z"},{"id":"KZhxmuWT","name":"Ottawa","slug":"ottawa","shortSlug":"ott","lnglat":[-75.69812,45.41117],"defaultSearchRadius":25000,"timezone":"America/Montreal","status":"active","creditValue":2500,"currencyCode":"CAD","region":"Ontario","regionCode":"ON","country":"Canada","countryCode":"CA","sortOrder":10,"locationCount":2,"_vid":5,"createdAt":"2014-09-30T21:46:23.773Z","updatedAt":"2017-03-01T14:53:14.423Z"}]
cities_df = pd.DataFrame(cities)
cities_df

,_vid,country,countryCode,createdAt,creditValue,currencyCode,defaultSearchRadius,id,lnglat,locationCount,name,region,regionCode,shortSlug,slug,sortOrder,status,timezone,updatedAt
0,79,United States,US,2014-02-05T16:32:47.014Z,4500,USD,25000.0,8jLq7Yxb3R,"[-73.9910835, 40.7358633]",149,New York City,New York,NY,nyc,new-york,1,active,America/New_York,2018-08-10T19:14:00.881Z
1,45,United States,US,2014-05-29T16:32:47.014Z,4500,USD,25000.0,kMG2IsYtKL,"[-122.404261, 37.784486]",65,SF Bay Area,California,CA,sf,san-francisco,2,active,America/Los_Angeles,2018-09-17T18:14:00.481Z
2,21,United States,US,2015-01-08T16:00:00.000Z,4500,USD,25000.0,lIRf1YeVcA,"[-71.106884, 42.370093]",32,Boston,Massachusetts,MA,bos,boston,3,active,America/New_York,2018-04-30T20:14:00.859Z
3,39,United States,US,2015-06-26T18:53:59.775Z,4500,USD,50000.0,fTb7f2cerf,"[-118.2436849, 34.0522342]",38,Los Angeles,California,CA,la,los-angeles,4,active,America/Los_Angeles,2018-06-29T18:14:01.592Z
4,17,United States,US,2015-06-26T18:53:59.816Z,4500,USD,50000.0,Zn3FHU9Kej,"[-87.6297982, 41.8781136]",29,Chicago,Illinois,IL,chi,chicago,5,active,America/Chicago,2018-08-30T18:14:00.484Z
5,16,United States,US,2015-06-26T18:53:59.774Z,4500,USD,NaN,jxCwZvcWuf,"[-77.0368707, 38.9071923]",27,Washington DC,District of Columbia,DC,dc,washington-dc,6,active,America/New_York,2017-12-01T19:14:00.712Z
6,29,United Kingdom,GB,2015-06-26T18:53:59.802Z,3500,GBP,NaN,HEG8fNPoN3,"[-0.1277583, 51.5073509]",40,London,England,England,lon,london,7,active,Europe/London,2018-03-12T17:14:00.878Z
7,40,Canada,CA,2013-10-15T15:36:34.294Z,2500,CAD,25000.0,x7bPMpzIvM,"[-73.588368, 45.503746]",31,Montreal,Québec,QC,mtl,montreal,8,active,America/Montreal,2018-02-26T17:14:00.073Z
8,34,Canada,CA,2015-12-11T15:50:32.779Z,2500,CAD,50000.0,e9447930a01e11e581aa851ef65f5f58,"[-79.38166, 43.6525]",43,Toronto,Ontario,ON,tor,toronto,9,active,America/Toronto,2018-06-23T03:14:00.462Z
9,5,Canada,CA,2014-09-30T21:46:23.773Z,2500,CAD,25000.0,KZhxmuWT,"[-75.69812, 45.41117]",2,Ottawa,Ontario,ON,ott,ottawa,10,active,America/Montreal,2017-03-01T14:53:14.423Z


In [68]:
# getting all we need for the analysis:
locations_data = cities_df[['locationCount','name','region','slug','currencyCode','countryCode']]
locations_data

,locationCount,name,region,slug,currencyCode,countryCode
0,149,New York City,New York,new-york,USD,US
1,65,SF Bay Area,California,san-francisco,USD,US
2,32,Boston,Massachusetts,boston,USD,US
3,38,Los Angeles,California,los-angeles,USD,US
4,29,Chicago,Illinois,chicago,USD,US
5,27,Washington DC,District of Columbia,washington-dc,USD,US
6,40,London,England,london,GBP,GB
7,31,Montreal,Québec,montreal,CAD,CA
8,43,Toronto,Ontario,toronto,CAD,CA
9,2,Ottawa,Ontario,ottawa,CAD,CA


In [70]:
#creating currency dictionaty with slug and currencyCode to calculate prices in USD per location. 
currency_dict = {}
n=0
for slug in locations_data['slug']:
    currency_dict[slug] = locations_data.loc[n]['currencyCode']
    n = n+1

In [71]:
currency_dict

{'boston': 'USD',
 'chicago': 'USD',
 'london': 'GBP',
 'los-angeles': 'USD',
 'montreal': 'CAD',
 'new-york': 'USD',
 'ottawa': 'CAD',
 'san-francisco': 'USD',
 'toronto': 'CAD',
 'washington-dc': 'USD'}

In [72]:
#OANDA currency on sep 23 2018 to calculate all values in USD
currency_change_dict = {'USD': 1, 'CAD': 1.29112, 'GBP': 0.76427}

In [73]:
index = ['boston', 'chicago', 'london', 'los-angeles', 'montreal', 'new-york', 'ottawa', 'san-francisco', 
         'toronto', 'washington-dc']
currencies = pd.DataFrame(currency_dict, index = index)
currencies = currencies.transpose()
currencies = currencies[['boston']]
currencies.rename(columns={'boston':'currency'})

,currency
boston,USD
chicago,USD
london,GBP
los-angeles,USD
montreal,CAD
new-york,USD
ottawa,CAD
san-francisco,USD
toronto,CAD
washington-dc,USD


In [74]:
# lets define our index for all dataframes to come:
urls_ids = price_analysis.index

In [75]:
currency_per_url = {}
for url in urls_ids:
    currency_per_url[url] = currency_dict[price_analysis.loc[url]['location']]

In [76]:
currency_per_url_df = pd.DataFrame(currency_per_url, index=urls_ids).transpose()[['https://breather.com/locations/boston/33-broad']]
currency_per_url_df.rename(columns={'https://breather.com/locations/boston/33-broad':'currency'}, inplace=True)
currency_per_url_df
price_analysis = price_analysis.merge(currency_per_url_df, left_index=True,right_index=True)

In [77]:
hourly_price_usd_list = []
daily_price_usd_list = []
for url in urls_ids:
    hourly_price_usd = price_analysis.loc[url]['hourly_price'] / currency_change_dict[price_analysis.loc[url]['currency']]
    hourly_price_usd_list.append(hourly_price_usd)
    daily_price_usd = price_analysis.loc[url]['daily_price'] / currency_change_dict[price_analysis.loc[url]['currency']]
    daily_price_usd_list.append(daily_price_usd)

In [78]:
price_analysis['hourly_price_usd'] = hourly_price_usd_list
price_analysis['daily_price_usd'] = daily_price_usd_list

In [81]:
# results:
price_analysis.head(3)

,hourly_price,daily_price,location,currency,hourly_price_usd,daily_price_usd
https://breather.com/locations/boston/262-washington-2,57.0,452.0,boston,USD,57.0,452.0
https://breather.com/locations/boston/50-congress-543,49.0,392.0,boston,USD,49.0,392.0
https://breather.com/locations/boston/11-beacon-605,65.0,520.0,boston,USD,65.0,520.0


In [84]:
# lets have a sneak peak on how avg prices change by location
prices_to_analyze = price_analysis[['location', 'hourly_price_usd', 'daily_price_usd']]
grouped_location = prices_to_analyze.groupby('location')
grouped_location.describe()

daily_price_usd                                                  \
                        count        mean         std         min         25%   
location                                                                        
boston                   29.0  793.724138  352.490618  312.000000  472.000000   
chicago                  28.0  676.571429  266.305788  352.000000  440.000000   
london                   30.0  994.064052  421.044163  418.700198  628.050296   
los-angeles              35.0  757.942857  234.528556  384.000000  624.000000   
montreal                 31.0  367.572840  128.699375  123.923415  309.808538   
new-york                144.0  854.222222  524.308728  240.000000  440.000000   
ottawa                    2.0  139.413842   21.906772  123.923415  131.668629   
san-francisco            66.0  885.818182  561.204060  248.000000  456.000000   
toronto                  41.0  628.986896  379.363362  216.865977  371.770246   
washington-dc            17.0  768.941176  312.536492  328.000000  552.000000   

                                                    hourly_price_usd  \
                      50%          75%          max            count   
location                                                               
boston         759.000000  1047.000000  1527.000000             30.0   
chicago        604.000000   876.000000  1400.000000             28.0   
london         942.075445  1203.763068  1936.488414             30.0   
los-angeles    736.000000   936.000000  1216.000000             35.0   
montreal       377.966417   464.712807   526.674515             31.0   
new-york       720.000000  1078.000000  2400.000000            144.0   
ottawa         139.413842   147.159056   154.904269              2.0   
san-francisco  712.000000  1200.000000  2536.000000             66.0   
toronto        514.282174   774.521346  1703.946961             41.0   
washington-dc  632.000000   920.000000  1432.000000             17.0   

                                                                        \
                     mean        std        min        25%         50%   
location                                                                 
boston          98.333333  43.650914  39.000000  60.500000   92.000000   
chicago         84.571429  33.288223  44.000000  55.000000   75.500000   
london         124.258007  52.630520  52.337525  78.506287  117.759431   
los-angeles     94.742857  29.316070  48.000000  78.000000   92.000000   
montreal        45.946605  16.087422  15.490427  38.726067   47.245802   
new-york       106.777778  65.538591  30.000000  55.000000   90.000000   
ottawa          17.426730   2.738346  15.490427  16.458579   17.426730   
san-francisco  110.727273  70.150508  31.000000  57.000000   89.000000   
toronto         78.623362  47.420420  27.108247  46.471281   64.285272   
washington-dc   96.117647  39.067061  41.000000  69.000000   79.000000   

                                       
                      75%         max  
location                               
boston         128.000000  191.000000  
chicago        109.500000  175.000000  
london         150.470384  242.061052  
los-angeles    117.000000  152.000000  
montreal        58.089101   65.834314  
new-york       134.750000  300.000000  
ottawa          18.394882   19.363034  
san-francisco  150.000000  317.000000  
toronto         96.815168  212.993370  
washington-dc  115.000000  179.000000

## Breather.com page source data:
as I said before, breather had in their page source all data classifications that we are going to analyze, so we are using that as a base for our dataframes.:

In [85]:
#cities= [{"id":"8jLq7Yxb3R","name":"New York City","slug":"new-york","shortSlug":"nyc","lnglat":[-73.9910835,40.7358633],"defaultSearchRadius":25000,"timezone":"America/New_York","status":"active","creditValue":4500,"currencyCode":"USD","region":"New York","regionCode":"NY","country":"United States","countryCode":"US","sortOrder":1,"locationCount":149,"_vid":79,"createdAt":"2014-02-05T16:32:47.014Z","updatedAt":"2018-08-10T19:14:00.881Z"},{"id":"kMG2IsYtKL","name":"SF Bay Area","slug":"san-francisco","shortSlug":"sf","lnglat":[-122.404261,37.784486],"defaultSearchRadius":25000,"timezone":"America/Los_Angeles","status":"active","creditValue":4500,"currencyCode":"USD","region":"California","regionCode":"CA","country":"United States","countryCode":"US","sortOrder":2,"locationCount":65,"_vid":45,"createdAt":"2014-05-29T16:32:47.014Z","updatedAt":"2018-09-17T18:14:00.481Z"},{"id":"lIRf1YeVcA","name":"Boston","slug":"boston","shortSlug":"bos","lnglat":[-71.106884,42.370093],"defaultSearchRadius":25000,"timezone":"America/New_York","status":"active","creditValue":4500,"currencyCode":"USD","region":"Massachusetts","regionCode":"MA","country":"United States","countryCode":"US","sortOrder":3,"locationCount":32,"_vid":21,"createdAt":"2015-01-08T16:00:00.000Z","updatedAt":"2018-04-30T20:14:00.859Z"},{"id":"fTb7f2cerf","name":"Los Angeles","slug":"los-angeles","shortSlug":"la","lnglat":[-118.2436849,34.0522342],"defaultSearchRadius":50000,"timezone":"America/Los_Angeles","status":"active","creditValue":4500,"currencyCode":"USD","region":"California","regionCode":"CA","country":"United States","countryCode":"US","sortOrder":4,"locationCount":38,"_vid":39,"createdAt":"2015-06-26T18:53:59.775Z","updatedAt":"2018-06-29T18:14:01.592Z"},{"id":"Zn3FHU9Kej","name":"Chicago","slug":"chicago","shortSlug":"chi","lnglat":[-87.6297982,41.8781136],"defaultSearchRadius":50000,"timezone":"America/Chicago","status":"active","creditValue":4500,"currencyCode":"USD","region":"Illinois","regionCode":"IL","country":"United States","countryCode":"US","sortOrder":5,"locationCount":29,"_vid":17,"createdAt":"2015-06-26T18:53:59.816Z","updatedAt":"2018-08-30T18:14:00.484Z"},{"id":"jxCwZvcWuf","name":"Washington DC","slug":"washington-dc","shortSlug":"dc","lnglat":[-77.0368707,38.9071923],"timezone":"America/New_York","status":"active","creditValue":4500,"currencyCode":"USD","region":"District of Columbia","regionCode":"DC","country":"United States","countryCode":"US","sortOrder":6,"locationCount":27,"_vid":16,"createdAt":"2015-06-26T18:53:59.774Z","updatedAt":"2017-12-01T19:14:00.712Z"},{"id":"HEG8fNPoN3","name":"London","slug":"london","shortSlug":"lon","lnglat":[-0.1277583,51.5073509],"timezone":"Europe/London","status":"active","creditValue":3500,"currencyCode":"GBP","region":"England","regionCode":"England","country":"United Kingdom","countryCode":"GB","sortOrder":7,"locationCount":40,"_vid":29,"createdAt":"2015-06-26T18:53:59.802Z","updatedAt":"2018-03-12T17:14:00.878Z"},{"id":"x7bPMpzIvM","name":"Montreal","slug":"montreal","shortSlug":"mtl","lnglat":[-73.588368,45.503746],"defaultSearchRadius":25000,"timezone":"America/Montreal","status":"active","creditValue":2500,"currencyCode":"CAD","region":"Québec","regionCode":"QC","country":"Canada","countryCode":"CA","sortOrder":8,"locationCount":31,"_vid":40,"createdAt":"2013-10-15T15:36:34.294Z","updatedAt":"2018-02-26T17:14:00.073Z"},{"id":"e9447930a01e11e581aa851ef65f5f58","name":"Toronto","slug":"toronto","shortSlug":"tor","lnglat":[-79.38166,43.6525],"defaultSearchRadius":50000,"timezone":"America/Toronto","status":"active","creditValue":2500,"currencyCode":"CAD","region":"Ontario","regionCode":"ON","country":"Canada","countryCode":"CA","sortOrder":9,"locationCount":43,"_vid":34,"createdAt":"2015-12-11T15:50:32.779Z","updatedAt":"2018-06-23T03:14:00.462Z"},{"id":"KZhxmuWT","name":"Ottawa","slug":"ottawa","shortSlug":"ott","lnglat":[-75.69812,45.41117],"defaultSearchRadius":25000,"timezone":"America/Montreal","status":"active","creditValue":2500,"currencyCode":"CAD","region":"Ontario","regionCode":"ON","country":"Canada","countryCode":"CA","sortOrder":10,"locationCount":2,"_vid":5,"createdAt":"2014-09-30T21:46:23.773Z","updatedAt":"2017-03-01T14:53:14.423Z"}]
use_cases= [{"slug":"offsite","label":"Team Offsite","synonyms":["Offsite"]},{"slug":"team-meeting","label":"Team Meeting","synonyms":[""]},{"slug":"presentation","label":"Presentation","synonyms":[""]},{"slug":"client-meeting","label":"Client Meeting","synonyms":[""]},{"slug":"consultation","label":"Therapy / Consultation","synonyms":["Therapy","Consulation"]},{"slug":"photography","label":"Photoshoot","synonyms":["Photography"]},{"slug":"phone","label":"Private Phone Call","synonyms":["Phonebooth","Telephone"]},{"slug":"solo","label":"Individual Work","synonyms":["Solo","Desk","Office"]},{"slug":"small-event","label":"Small Event","synonyms":[""]},{"slug":"workshop","label":"Class / Workshop","synonyms":[""]}]
amenities2= [{"label":"Whiteboard","slug":"whiteboard","siblings":[],"index":0},{"label":"Premium building","slug":"premium-building","siblings":[],"index":1},{"label":"Parking","slug":"parking","siblings":[],"index":2},{"label":"Speakers","slug":"speakers","siblings":[],"index":3},{"label":"On-site catering","slug":"on-site-catering","siblings":[],"index":4},{"label":"Fireplace","slug":"fireplace","siblings":[],"index":5},{"label":"Projector","slug":"projector","siblings":["flatscreen"],"index":6},{"label":"Projector Screen","slug":"projector-screen","siblings":[],"index":7},{"label":"Flatscreen TV","slug":"flatscreen","siblings":["projector"],"index":8},{"label":"Apple TV","slug":"apple-tv","siblings":[],"index":9},{"label":"Google Chromebox","slug":"google-chromebox","siblings":[],"index":10},{"label":"wePresent","slug":"wepresent","siblings":[],"index":11},{"label":"Airtame","slug":"airtame","siblings":[],"index":12},{"label":"Webcam","slug":"webcam","siblings":[],"index":13},{"label":"Air conditioning","slug":"air-conditioning","siblings":[],"index":14},{"label":"Free coffee","slug":"coffee","siblings":[],"index":15},{"label":"Tea point","slug":"tea","siblings":[],"index":16},{"label":"Free water","slug":"water","siblings":[],"index":17},{"label":"Water for sale","slug":"water-for-sale","siblings":[],"index":18},{"label":"Mini fridge","slug":"mini-fridge","siblings":["kitchen","ensuite-kitchen"],"index":19},{"label":"Dishwasher","slug":"dishwasher","siblings":[],"index":20},{"label":"Microwave","slug":"microwave","siblings":[],"index":21},{"label":"Skylight","slug":"skylight","siblings":[],"index":22},{"label":"WiFi","slug":"wifi","siblings":[],"index":23},{"label":"Ensuite bathroom","slug":"ensuite-restroom","siblings":[],"index":24},{"label":"Ensuite kitchenette","slug":"ensuite-kitchen","siblings":["kitchen","mini-fridge"],"index":25},{"label":"Waiting room","slug":"waiting-room","siblings":[],"index":26},{"label":"Kitchenette on premises","slug":"kitchen","siblings":["mini-fridge","ensuite-kitchen"],"index":27},{"label":"Blinds","slug":"curtains","siblings":[],"index":28},{"label":"Lobby staff","slug":"doorman","siblings":[],"index":29},{"label":"Great view","slug":"great-view","siblings":[],"index":30},{"label":"Wheelchair accessible","slug":"wheelchair-accessible","siblings":[],"index":31},{"label":"Mirror","slug":"mirror","siblings":[],"index":32},{"label":"Yoga mat","slug":"yoga-mat","siblings":[],"index":33},{"label":"Elevator access","slug":"elevator-access","siblings":[],"index":34},{"label":"Phone charger","slug":"phone-charger","siblings":[],"index":35},{"label":"Bathroom on premises","slug":"restroom","siblings":[],"index":36},{"label":"HDMI","slug":"hdmi","siblings":[],"index":37},{"label":"Thunderbolt","slug":"thunderbolt","siblings":[],"index":38},{"label":"Wireless presentation","slug":"presentation","siblings":[],"index":39},{"label":"USB-C Multiport","slug":"usb-adapter","siblings":[],"index":40},{"label":"Catering allowed","slug":"catering-allowed","siblings":[],"index":41}]

In [86]:
#cities_df = pd.DataFrame(cities)
use_cases_df = pd.DataFrame(use_cases)
amenities_df = pd.DataFrame(amenities2)

### Logics:
Cities data I'm going to use to see how many locations we were able to grab compared to what its available. All url lists were created based on a day in march with the most amount of hrefs available.

Amenities data will be used to build the columns for the amenities DataFrame (rows:  urls, columns: amenities, values: 1 if True, 0 if False. 

Use cases  will be used as amenities but for the good for section. (note that good for might have wrong values (that wer tached off), so I might not work on this data.

In [88]:
amenities_columns = amenities_df['label'].tolist
amenities_list = amenities_df['label']

In [90]:
# get all amenities per url, and see if it matches with the amenities we have in our list.
amenities_booleans_dict = {}
for amenity in amenities_list:
    amenities_booleans_dict[amenity] = {}
    for url in urls_ids:
        amenities_per_url = pd.Series(primary_df.loc[url]['amenities'])
        boolean = amenities_per_url.isin([amenity])
        #this will not give me contains, but for now we'll use it like this. 
        n = 0
        for dummy in boolean:
            if dummy == True:
                n = n+1
            else:
                n = n
        amenities_booleans_dict[amenity][url] = n

In [91]:
amenities_booleans_df = pd.DataFrame(amenities_booleans_dict, index=urls_ids)
amenities_booleans_df.head(5)

,Air conditioning,Airtame,Apple TV,Bathroom on premises,Blinds,Catering allowed,Dishwasher,Elevator access,Ensuite bathroom,Ensuite kitchenette,...,USB-C Multiport,Waiting room,Water for sale,Webcam,Wheelchair accessible,Whiteboard,WiFi,Wireless presentation,Yoga mat,wePresent
url_id,,,,,,,,,,,,,,,,,,,,,
https://breather.com/locations/boston/262-washington-2,1,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
https://breather.com/locations/boston/50-congress-543,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
https://breather.com/locations/boston/11-beacon-605,1,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
https://breather.com/locations/boston/262-washington-8,1,0,0,1,1,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
https://breather.com/locations/boston/11-beacon-1110,1,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0


## Finally, we'll assamble the final dataframe to be saved and used for analysis

In [92]:
final_dataset = pd.merge(price_analysis[['location','currency','hourly_price_usd','daily_price_usd']], accommodates, left_index = True, right_index = True)
final_dataset = final_dataset.merge(ratings_all, left_index = True, right_index = True)
final_dataset= final_dataset.merge(amenities_booleans_df, left_index = True, right_index = True)
final_dataset.head(3)

,location,currency,hourly_price_usd,daily_price_usd,at_tables,in_lounge,maximum_total,sq_ft,ratings,rating_score,...,USB-C Multiport,Waiting room,Water for sale,Webcam,Wheelchair accessible,Whiteboard,WiFi,Wireless presentation,Yoga mat,wePresent
url_id,,,,,,,,,,,,,,,,,,,,,
https://breather.com/locations/boston/262-washington-2,boston,USD,57.0,452.0,8.0,3.0,11.0,440.0,29.0,4.7,...,1,0,0,0,0,0,1,0,0,0
https://breather.com/locations/boston/50-congress-543,boston,USD,49.0,392.0,4.0,2.0,6.0,226.0,395.0,4.4,...,0,0,0,0,0,1,1,0,0,0
https://breather.com/locations/boston/11-beacon-605,boston,USD,65.0,520.0,12.0,2.0,14.0,485.0,9.0,4.6,...,1,0,0,0,0,0,1,0,0,0
